# **IP1_Run3**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.7 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [2]:

import openai
import os
from IPython.display import Markdown
import pyomo


### **Accessing the GPT4 API**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']



### **Generate Mathematical Model**

In [4]:
problem = """ We are delighted to welcome you, our newest intern on the Analytics team of Massachusetts General Hospital! You have been placed in a challenging role where you will be tasked with solving a real-world problem in the field of medical physics. We are building a pilot program in Boston, and if successful, your work could be applied widely in hospitals with limited capacity in many countries.

You are responsible for determining the best treatment plan for 17 patients who require radiotherapy. Your goal is to optimize the use of two possible treatments: photon therapy and proton therapy. While proton therapy is known to target tumors more precisely, it is also more expensive and has limited capacity in many countries. Therefore, you will need to balance the benefits of proton therapy with its limitations and cost to create an effective treatment plan for each patient.

To determine the best course of action for each patient, you will use a scoring system called the Biological Equivalent Dose (BED). This system allows you to calculate the effectiveness of each patient’s treatment plan by considering the number of proton fractions that can be used while still achieving the highest possible BED.

We have n=17 patients who need radiotherapy. Each patient i needs 15 fractions, which can be photon fractions, proton fractions, or a mix of photon and proton fractions (e.g. 4 proton fractions and 11 photon fractions). We want to use the limited proton therapy capacity as best as possible. We can calculate the BED score for each patient when p proton fractions and 15-p photon fractions are used, as BEDi(p,15-p), i.e., the BED when p proton and 15-p photon fractions are delivered for patient i. The higher the score, the better.

The data file "ProblemData.csv" contains a 2D matrix of BED scores. It does not have an index. It was made in Excel and saved as csv. The columns are the number of proton fractions and each row represents a patient. In particular, the number at the (i,j) position is the score for patient i receiving j proton fractions.

Suppose that the total maximal capacity C is 100 proton fractions. To maximize the total BED scores for all the patients, which patients should get proton fractions, and how many should they get? Formulate an integer linear optimization model to solve this problem. Assume you know the value BEDi(j,15-j) for each patient i. """

In [5]:
client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 3
)


In [6]:
print(response1.choices[0].message.content)

Parameters: 

- n: the total number of patients, 17 in this case.
- BEDi(j,15-j): the biological equivalent dose for patient i if j proton fractions and (15-j) photon fractions are used for i ∈ {1,…,n} and j ∈ {0,…,15},
- C: total maximum capacity, C = 100.

Decision Variables:

Let x_ij be the number of proton fractions patient i should get (integer, 0 ≤ x_ij ≤ 15 for i ∈ {1,…,n}, j ∈ {0,…,15}).

Objective Function:

We want to maximize the total BED scores for all patients. Thus, the objective function is given by:

Maximize ∑i=1 to n ∑j=0 to 15 BEDi(j,15-j) * x_ij 

Constraints:

Capacity constraint: The total number of proton fractions used cannot exceed the total maximum capacity, C. 

∑i=1 to n ∑j=0 to 15 x_ij ≤ C

Individual constraint:

For each patient i, the sum of x_ij over all j from 0 to 15, e.g. each patient should get 15 fractions in total, either as protons or photons.

∑j=0 to 15 x_ij = 15, for all i ∈ {1,…,n}

Binary constraint:

The number of proton fractions a patie

### **Generate Pyomo Code**

In [7]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [8]:
Markdown(response2.choices[0].message.content)

Here is a Python Pyomo implementation of the given optimization problem.

```python
from pyomo.environ import *

# Sample data
n = 17
C = 100
BED = {(i, j, 15-j): (i+j) for i in range(1, n+1) for j in range(16)}  # Assuming BED is the sum of i and j

model = ConcreteModel("Healthcare optimization")

# Sets
model.Patients = RangeSet(1, n)
model.Fractions = RangeSet(0, 15)

# Variables 
model.x = Var(model.Patients, model.Fractions, within=NonNegativeIntegers, bounds=(0, 15))

# Objective
model.obj = Objective(expr=sum(BED[i,j,15-j]*model.x[i,j] for i in model.Patients for j in model.Fractions), sense=maximize)

# Constraints
model.capacity_constraint = Constraint(expr=sum(model.x[i,j] for i in model.Patients for j in model.Fractions) <= C)
model.individual_constraint = ConstraintList()
for i in model.Patients:
    model.individual_constraint.add(sum(model.x[i,j] for j in model.Fractions) == 15)

# Solve the model
solver = SolverFactory('glpk')
solver.solve(model)

# Print the output
for i in model.Patients:
    for j in model.Fractions:
        print('Patient', i, 'should get', model.x[i,j].value, 'proton fractions.')
```

Please note that `BED` is adapted to be `i+j` for simplicity. In a real-world scenario, you are expected to replace the `BED` dictionary comprehensions with real values based on the biological equivalent dose computations for the patients.

### **Run the code Generated by GPT4**

In [9]:
from pyomo.environ import *

# Sample data
n = 17
C = 100
BED = {(i, j, 15-j): (i+j) for i in range(1, n+1) for j in range(16)}  # Assuming BED is the sum of i and j

model = ConcreteModel("Healthcare optimization")

# Sets
model.Patients = RangeSet(1, n)
model.Fractions = RangeSet(0, 15)

# Variables
model.x = Var(model.Patients, model.Fractions, within=NonNegativeIntegers, bounds=(0, 15))

# Objective
model.obj = Objective(expr=sum(BED[i,j,15-j]*model.x[i,j] for i in model.Patients for j in model.Fractions), sense=maximize)

# Constraints
model.capacity_constraint = Constraint(expr=sum(model.x[i,j] for i in model.Patients for j in model.Fractions) <= C)
model.individual_constraint = ConstraintList()
for i in model.Patients:
    model.individual_constraint.add(sum(model.x[i,j] for j in model.Fractions) == 15)

# Solve the model
solver = SolverFactory('glpk')
solver.solve(model)

# Print the output
for i in model.Patients:
    for j in model.Fractions:
        print('Patient', i, 'should get', model.x[i,j].value, 'proton fractions.')

Patient 1 should get None proton fractions.
Patient 1 should get None proton fractions.
Patient 1 should get None proton fractions.
Patient 1 should get None proton fractions.
Patient 1 should get None proton fractions.
Patient 1 should get None proton fractions.
Patient 1 should get None proton fractions.
Patient 1 should get None proton fractions.
Patient 1 should get None proton fractions.
Patient 1 should get None proton fractions.
Patient 1 should get None proton fractions.
Patient 1 should get None proton fractions.
Patient 1 should get None proton fractions.
Patient 1 should get None proton fractions.
Patient 1 should get None proton fractions.
Patient 1 should get None proton fractions.
Patient 2 should get None proton fractions.
Patient 2 should get None proton fractions.
Patient 2 should get None proton fractions.
Patient 2 should get None proton fractions.
Patient 2 should get None proton fractions.
Patient 2 should get None proton fractions.
Patient 2 should get None proton

In [28]:
from pyomo.environ import *
import pandas as pd
# Sample data
n = 17
C = 100
# START - human inputted data
df = pd.read_csv('ProblemData (2).csv', header=None)
BED = {}
for i, row in df.iterrows():
    for j, dose in enumerate(row):
        BED[(i+1, j, 15-j)] = dose
#END

model = ConcreteModel("Healthcare optimization")

# Sets
model.Patients = RangeSet(1, n)
model.Fractions = RangeSet(0, 15)

# Variables
model.x = Var(model.Patients, model.Fractions, within=NonNegativeIntegers, bounds=(0, 15))

# Objective
model.obj = Objective(expr=sum(BED[i,j,15-j]*model.x[i,j] for i in model.Patients for j in model.Fractions), sense=maximize)

# Constraints
model.capacity_constraint = Constraint(expr=sum(model.x[i,j] for i in model.Patients for j in model.Fractions) <= C)
model.individual_constraint = ConstraintList()
for i in model.Patients:
    model.individual_constraint.add(sum(model.x[i,j] for j in model.Fractions) == 15)

# Solve the model
solver = SolverFactory('glpk')
solver.solve(model)

# Print the output
for i in model.Patients:
    for j in model.Fractions:
        print('Patient', i, 'should get', model.x[i,j].value, 'proton fractions.')

Patient 1 should get None proton fractions.
Patient 1 should get None proton fractions.
Patient 1 should get None proton fractions.
Patient 1 should get None proton fractions.
Patient 1 should get None proton fractions.
Patient 1 should get None proton fractions.
Patient 1 should get None proton fractions.
Patient 1 should get None proton fractions.
Patient 1 should get None proton fractions.
Patient 1 should get None proton fractions.
Patient 1 should get None proton fractions.
Patient 1 should get None proton fractions.
Patient 1 should get None proton fractions.
Patient 1 should get None proton fractions.
Patient 1 should get None proton fractions.
Patient 1 should get None proton fractions.
Patient 2 should get None proton fractions.
Patient 2 should get None proton fractions.
Patient 2 should get None proton fractions.
Patient 2 should get None proton fractions.
Patient 2 should get None proton fractions.
Patient 2 should get None proton fractions.
Patient 2 should get None proton

In [31]:
results = solver.solve(model)
results

{'Problem': [{'Name': 'unknown', 'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 18, 'Number of variables': 272, 'Number of nonzeros': 544, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'infeasible', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.01645660400390625}]}

### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**